In [14]:
!uv pip install plotly pyvis


Resolved 26 packages in 661ms                                        
Prepared 2 packages in 424ms                                             
Installed 2 packages in 5ms                                 
 + jsonpickle==4.1.1
 + pyvis==0.3.2


In [9]:
!uv pip install -U kaleido

Resolved 13 packages in 371ms                                        
Audited 13 packages in 0.17ms


In [12]:
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

# --- 1. Generate Sample Data ---
num_vectors = 300
vector_dim = 64
data = np.float32(np.random.rand(num_vectors, vector_dim))

# --- 2. Project High-Dimensional Data to 2D for Visualization ---
print("Running t-SNE to get 2D coordinates...")
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
xy_coords = tsne.fit_transform(data)

# --- 3. Simulate HNSW Layer Structure ---
nodes_by_layer = {0: list(range(num_vectors))}
p_layer1, p_layer2 = 0.2, 0.25

nodes_by_layer[1] = [i for i in nodes_by_layer[0] if np.random.rand() < p_layer1]
nodes_by_layer[2] = [i for i in nodes_by_layer[1] if np.random.rand() < p_layer2]

print(f"Layer 0 nodes: {len(nodes_by_layer[0])}")
print(f"Layer 1 nodes: {len(nodes_by_layer[1])}")
print(f"Layer 2 nodes: {len(nodes_by_layer[2])}")

# --- 4. Create Edges for each layer based on proximity in 2D space ---
edges_by_layer = {}
connections_per_node = {0: 7, 1: 5, 2: 3}

for layer_num, node_ids in nodes_by_layer.items():
    if len(node_ids) < 2:
        edges_by_layer[layer_num] = []
        continue

    layer_coords = xy_coords[node_ids]
    k = min(len(node_ids), connections_per_node[layer_num] + 1)
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(layer_coords)
    distances, indices = nbrs.kneighbors(layer_coords)

    edges = []
    for i, neighbors in enumerate(indices):
        original_node_id = node_ids[i]
        for neighbor_idx in neighbors[1:]:
            edges.append((original_node_id, node_ids[neighbor_idx]))
    edges_by_layer[layer_num] = edges

# --- 5. Plot with Plotly ---
fig = go.Figure()
colors = {0: 'lightgrey', 1: 'rgba(255,165,0,0.7)', 2: 'rgba(255,0,0,0.8)'}
node_sizes = {0: 3, 1: 5, 2: 7}

# Add edges
for layer_num in sorted(edges_by_layer.keys()):
    edge_x, edge_y, edge_z = [], [], []
    for start_node, end_node in edges_by_layer[layer_num]:
        edge_x += [xy_coords[start_node][0], xy_coords[end_node][0], None]
        edge_y += [xy_coords[start_node][1], xy_coords[end_node][1], None]
        edge_z += [layer_num, layer_num, None]

    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color=colors[layer_num], width=layer_num * 2 + 1),
        name=f'Layer {layer_num} Edges'
    ))

# Add nodes
for layer_num in sorted(nodes_by_layer.keys(), reverse=True):
    node_ids = nodes_by_layer[layer_num]
    fig.add_trace(go.Scatter3d(
        x=xy_coords[node_ids, 0],
        y=xy_coords[node_ids, 1],
        z=[layer_num] * len(node_ids),
        mode='markers',
        marker=dict(size=node_sizes[layer_num], color=colors[layer_num], symbol='circle'),
        name=f'Layer {layer_num} Nodes'
    ))

fig.update_layout(
    title='3D Visualization of a Simulated HNSW Graph',
    scene=dict(
        xaxis_title='t-SNE Dim 1',
        yaxis_title='t-SNE Dim 2',
        zaxis_title='HNSW Layer'
    ),
    template='plotly_dark',
    margin=dict(r=10, l=10, b=10, t=40),
    showlegend=True
)

# --- 6. Show and Save the Plot as PNG ---
fig.show()


# Save interactive HTML file (no Kaleido needed)
fig.write_html("hnsw_graph.html")
print("✅ Interactive plot saved as 'hnsw_graph.html'. Open it in your browser.")

Running t-SNE to get 2D coordinates...
Layer 0 nodes: 300
Layer 1 nodes: 55
Layer 2 nodes: 17


✅ Interactive plot saved as 'hnsw_graph.html'. Open it in your browser.


In [11]:
import matplotlib.pyplot as plt

# Fallback static visualization (2D projection)
plt.figure(figsize=(8, 6))
for layer_num, edges in edges_by_layer.items():
    for a, b in edges:
        plt.plot(
            [xy_coords[a, 0], xy_coords[b, 0]],
            [xy_coords[a, 1], xy_coords[b, 1]],
            color='gray', alpha=0.3
        )
for layer_num, ids in nodes_by_layer.items():
    plt.scatter(
        xy_coords[ids, 0],
        xy_coords[ids, 1],
        s=(layer_num + 1) * 10,
        label=f'Layer {layer_num}',
        alpha=0.8
    )
plt.legend()
plt.title("2D Projection of Simulated HNSW Graph")
plt.xlabel("t-SNE Dim 1")
plt.ylabel("t-SNE Dim 2")
plt.savefig("hnsw_graph_fallback.png", dpi=300, bbox_inches='tight')
plt.close()
print("✅ Static fallback plot saved as 'hnsw_graph_fallback.png'")


✅ Static fallback plot saved as 'hnsw_graph_fallback.png'


In [15]:
from pyvis.network import Network

# --- 1. Create network ---
net = Network(
    height='700px',
    width='100%',
    bgcolor='#0d1117',
    font_color='white',
    directed=True  # arrows for hierarchy
)

# --- 2. Dummy hierarchical knowledge data ---
nodes = {
    "AI": ["Machine Learning", "Deep Learning", "NLP", "Computer Vision"],
    "Machine Learning": ["Supervised", "Unsupervised", "Reinforcement"],
    "Deep Learning": ["CNN", "RNN", "Transformers"],
    "NLP": ["LLMs", "Embeddings", "Text Classification"],
    "Computer Vision": ["Image Classification", "Object Detection", "Segmentation"],
}

# --- 3. Add nodes & edges ---
for parent, children in nodes.items():
    net.add_node(parent, label=parent, color='#ffcc00', size=25)
    for child in children:
        net.add_node(child, label=child, color='#03dac6', size=20)
        net.add_edge(parent, child, color='#888888')

# --- 4. Hierarchical layout ---
net.set_options("""
var options = {
  layout: {
    hierarchical: {
      direction: "UD",
      sortMethod: "hubsize"
    }
  },
  nodes: {
    shape: "dot",
    scaling: { min: 10, max: 30 }
  },
  edges: {
    smooth: true,
    arrows: { to: { enabled: true } }
  },
  physics: {
    enabled: false
  }
}
""")

# --- 5. Save and show ---
net.save_graph("hierarchical_knowledge_graph.html")
print("✅ Interactive 2D hierarchical graph saved as 'hierarchical_knowledge_graph.html'")


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)